In [1]:
import pandas as pd
import numpy as np
import os
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction as aF
from pathlib import Path

from pandas_ml import ConfusionMatrix
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, recall_score
from sklearn.decomposition import PCA 


from itertools import product

In [2]:
path_data = Path('../data/')
df_labels = labels = pd.read_csv(path_data/'ComParE2018_AtypicalAffect.txt', sep="\t")
df_labels['subset'] = df_labels['file_name'].str.split('_').apply(lambda x: x[0])
df_labels.head()


# & 
#                                    (df_labels['emotion'] == 'neutral')
X_train, y_train = ((df_labels.loc[(df_labels['subset']=='train')][['file_name']]), 
                    df_labels.loc[df_labels['subset']=='train']['emotion'])

X_devel, y_devel = (df_labels.loc[df_labels['subset']=='devel'][['file_name']], 
                    df_labels.loc[df_labels['subset']=='devel']['emotion'])

#train and development
X_train_all, y_train_all = (pd.concat([X_train, X_devel], axis=0).reset_index(), 
                            pd.concat([y_train, y_devel], axis=0).reset_index())


X_test, y_test = (df_labels.loc[df_labels['subset']=='test'][['file_name']], 
                    df_labels.loc[df_labels['subset']=='test']['emotion'])

X_train_all.head(), y_train_all.head()

(   index       file_name
 0      0  train_0001.wav
 1      1  train_0002.wav
 2      2  train_0003.wav
 3      3  train_0004.wav
 4      4  train_0005.wav,    index  emotion
 0      0  neutral
 1      1  neutral
 2      2  neutral
 3      3  neutral
 4      4  neutral)

In [3]:
class ReadRawData(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        X_c = X.copy()
        df_sample_data = X_c['file_name'].apply(lambda x: audioBasicIO.readAudioFile(path_data/'wav'/x))
        X_c['sample_rate'] = df_sample_data.apply(lambda x: x[0])
        X_c['raw_data'] = df_sample_data.apply(lambda x: x[1])
        X_c['duration'] = X_c['raw_data'].apply(lambda x: x.size)/X_c['sample_rate']
        return X_c
        
rrd = ReadRawData()
# X_train = rrd.fit_transform(X_train)
# X_devel =rrd.transform(X_devel)
X_train_all = rrd.fit_transform(X_train_all)
X_test =rrd.transform(X_test)
X_train_all.head()

,index,file_name,sample_rate,raw_data,duration
0,0,train_0001.wav,16000,"[8, -16, -121, -137, -27, 25, -6, -85, -84, 13...",2.149812
1,1,train_0002.wav,16000,"[364, 614, 507, 569, 466, 485, 458, 422, 401, ...",1.263750
2,2,train_0003.wav,16000,"[-15, -17, 21, 14, 18, 25, 22, 45, 49, 22, 27,...",0.817125
3,3,train_0004.wav,16000,"[-819, -1333, -1253, -1463, -1517, -1561, -164...",0.758125
4,4,train_0005.wav,16000,"[-100, -67, -311, -415, -351, -369, -362, -302...",0.770312


In [38]:
class PyAudioAnalysisFeaturesFactory(BaseEstimator, TransformerMixin):
    
    def __init__(self, 
                 limit_to=4, #parameter to control where to truncate (or pad)
                 pad=['resize', 'zero', 'mean', 'stat'][0], # what kind of padding to use
                 frame_size=0.05, #PyAudio default
                 frame_step=0.025,#PyAudio default
                 keep_duration=True #to keep duration as a feature
                ):
        self.limit_to, self.pad = limit_to, pad
        self.frame_size, self.frame_step = frame_size, frame_step
        self.keep_duration = keep_duration
        self.key = f'{limit_to}_{pad}_{frame_size}_{frame_step}_{keep_duration}'
        self.counter = 0

    
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X):
        X_c = X.copy()
        if self.pad == 'stat':
            features = X_c.apply(lambda x: self.add_features_functionals((x['sample_rate'], 
                                                                          x['raw_data'])), 
                                 axis=1)
            features = pd.DataFrame(features.values.tolist())
        else:
            features = X_c.apply(lambda x: self.add_features((x['sample_rate'], x['raw_data'])), axis=1)
            # these steps are just for getting the column names
            x = X_c['raw_data'].iloc[0]
            fs = X_c['sample_rate'].iloc[0]
            s, t = aF.stFeatureExtraction(np.resize(x,(fs*self.limit_to,)),
                                          fs,
                                          self.frame_size*fs,
                                          self.frame_step*fs)
            col_names = np.array([f'{name}_{i}' for i in range(s.shape[1]) for name in t]).reshape(-1)
            features = pd.DataFrame(features.values.tolist(), columns=col_names)
        
        if self.keep_duration:
            features['duration'] = X_c['duration']
        return features.astype(np.float32)
            
            
    def add_features(self, res):
        if self.counter%20 == 0:
            print(self.counter)
        self.counter += 1
        fs,x = res
        diff = (self.limit_to*fs)-x.shape[0]
        if self.pad == 'resize':
            x = np.resize(x,(fs*self.limit_to,))
        elif self.pad == 'zero':
            x = np.hstack([np.zeros(diff), x]) if diff>0 else x[:(self.limit_to*fs)]
        else:
            x = np.hstack([np.repeat(x.mean(), diff), x]) if diff>0 else x[:(self.limit_to*fs)]
            
        s,t = aF.stFeatureExtraction(x,fs,self.frame_size*fs,self.frame_step*fs)
        s = s.T.reshape(-1)
        return s
    
    def add_features_functionals(self, res):
        fs,x = res
        s,t = aF.stFeatureExtraction(x,fs,self.frame_size*fs,self.frame_step*fs)
        summarized_functionals = np.hstack([
            s.mean(axis=1), 
            s.min(axis=1), 
            s.max(axis=1), 
            s.std(axis=1)
        ])
        return summarized_functionals
    

In [ ]:
# feature_transform_params = {
#     'limit_to': [2, 4, 6],
#     'pad': ['resize', 'zero', 'mean']
# }



# feature_transform_param_combs = [dict(zip(feature_transform_params.keys(),v))
#                                  for v in product(*feature_transform_params.values())]
    

# feature_dict = dict()

# for i in feature_transform_param_combs:
#     print(i)
#     paaff = PyAudioAnalysisFeaturesFactory(**i)
#     feature_dict[paaff.key] = paaff.fit_transform(X_train_all)

paaff = PyAudioAnalysisFeaturesFactory(pad='resize')
paaff.fit_transform(X_train_all)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340


In [81]:
# storing the dictionary for future use

print(feature_dict.keys())

with open(path_data/'feature_dict.pickle', 'wb') as handle:
    pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open(path_data/'filename.pickle', 'rb') as handle:
#     feature_dict = pickle.load(handle)


# paf = PyAudioAnalysisFeatures()
# X_train = paf.transform(X_train)
# X_devel = paf.transform(X_devel)
# X_test = paf.transform(X_test)

# X_train.to_csv(path_data/'train_4-sec.csv')
# X_test.to_csv(path_data/'test_4-sec.csv')
# X_devel.to_csv(path_data/'devel_4-sec.csv')

# X_train = pd.read_csv(path_data/'train_4-sec.csv')
# X_test = pd.read_csv(path_data/'test_4-sec.csv')
# X_devel = pd.read_csv(path_data/'devel_4-sec.csv')


dict_keys(['2_resize_0.05_0.025_True', '2_zero_0.05_0.025_True', '2_mean_0.05_0.025_True', '4_resize_0.05_0.025_True', '4_zero_0.05_0.025_True', '4_mean_0.05_0.025_True', '6_resize_0.05_0.025_True', '6_zero_0.05_0.025_True', '6_mean_0.05_0.025_True'])

In [86]:
class PyAudioAnalysisFeatures(PyAudioAnalysisFeaturesFactory, BaseEstimator, TransformerMixin):
    
    def __init__(self, **kwargs):
        PyAudioAnalysisFeaturesFactory.__init__(self, **kwargs)
        
    def fit(self):
        return self
    
    def transform(self, X):
        X_base = feature_dict[self.key]
        return X_base.loc[X.index]

In [87]:
PyAudioAnalysisFeatures().transform(X_train.loc[[0,2,4]])

,zcr_0,energy_0,energy_entropy_0,spectral_centroid_0,spectral_spread_0,spectral_entropy_0,spectral_flux_0,spectral_rolloff_0,mfcc_1_0,mfcc_2_0,...,chroma_5_158,chroma_6_158,chroma_7_158,chroma_8_158,chroma_9_158,chroma_10_158,chroma_11_158,chroma_12_158,chroma_std_158,duration
0,0.227785,0.329161,0.456821,0.480601,0.488110,0.481852,0.401752,0.215269,0.101377,0.103880,...,0.020262,0.018331,0.019932,0.035767,0.031810,0.024497,0.028909,0.038841,0.039237,2.149812
2,0.075094,0.072591,0.092616,0.147685,0.200250,0.125156,0.052566,0.061327,0.060075,0.055069,...,0.015770,0.018118,0.031636,0.040933,0.051808,0.070929,0.050378,0.034263,0.028873,0.817125
4,0.125156,0.200250,0.245307,0.161452,0.077597,0.087610,0.106383,0.105131,0.071339,0.058824,...,0.058274,0.069391,0.061403,0.034582,0.050379,0.015063,0.019894,0.033224,0.041645,0.770312


In [30]:
model_pipeline = pipeline.Pipeline([
    ('features', PyAudioAnalysisFeatures()),
    ('pca', PCA(n_components=500)),
    ('model', AdaBoostClassifier(DecisionTreeClassifier(class_weight={'neutral': 100000, 
                                                                      'happy': 100, 
                                                                      'sad': 1, 
                                                                      'angry':1}), 
                                 n_estimators=100, learning_rate=10.0))
])

param_grid = {
    'pca__n_components': [15, 50, 100, 250, 500],
    'model__class_weight': [
                        {'neutral': 1, 'happy': 1, 'sad': 100000, 'angry':100000},
                        {'neutral': 100000, 'happy': 100000, 'sad': 1, 'angry':1}
                    ],
    'model__learning_rate': [0.1, 1, 10.0]
}

for k,v in feature_transform_params:
    param_grid['features__'+k] = v
    
# custom scoring - Unweighted Average Recall (from the paper),  
# because our data is imbalanced
uar = make_scorer(recall_score, average='macro')

model_pipeline_cv = GridSearchCV(
    estimator= model_pipeline, #use the model
    param_grid=param_grid, # generate combinations from the parameter grid
    scoring=uar, #UAR to pick the best params
    n_jobs=4, # use these many cores for faster parallel processing
    cv=5, # k fold cv
    refit=True, # refit the best parameters on all of the data
    verbose=50, #give detailed progress
    iid=True)

model_pipeline_cv.fit(X_train_all, y_train_all)

KeyboardInterrupt: 

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False):
    
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
    
def reporting(y_test,y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(cm, target_names=np.unique(y_train))
    print(confusion_matrix)
    print("\n",classification_report(y_test,y_pred))
    print(accuracy_score(y_test,y_pred))

In [ ]:
y_pred = model_pipeline.predict(X_test)

In [ ]:
reporting(y_test,y_pred)

In [ ]:
classification_report(y_test,y_pred)